### CS/ECE/ISyE 524 &mdash; Introduction to Optimization &mdash; Spring 2018 ###

# Feed To Others What You Would Eat Yourself: Creating Affordable and Interesting Menus for All Americans #

#### Anne Ulrich (aulrich3@wisc.edu) and Harish Veeramani (hveeramani@wisc.edu)

*****

### Table of Contents

1. [Introduction](#1.-Introduction)
1. [Mathematical Model](#2.-Mathematical-model)
1. [Solution](#3.-Solution)
1. [Results and Discussion](#4.-Results-and-discussion)
  1. [Optional Subsection](#4.A.-Feel-free-to-add-subsections)
1. [Conclusion](#5.-Conclusion)

## 1. Introduction ##

Here in America, commentators have long noted the structural factors creating permanent communities of underprivileged citizens. In recent years, we’ve seen a particular focus on lack of access to critical health care and prevention resources, with legislative actions and nonprofit initiatives ranging from the Affordable Care Act to Michelle Obama’s Let’s Move! campaign to reduce childhood obesity. One emerging area of investigation revolves around communities’ access to a variety high-quality, healthy foods. In our project, we aim to quantify these access issues by consider the optimal diets that can be constructed from the selection available at a variety of local different stores. 

In constructing our project, we realized that access to healthy nutritional foods depends on a variety of issues. How far away is the grocery store from your house? Do you even have a way of getting to the nearest grocery store? How can you construct a varied diet from the food available to you? Is it better to shop at more than one store given the variation in cost between stores? We sought to answer these questions by improving on a well-known study from economic literature: the Stigler diet.

George Stigler was an economist at Columbia University and the University of Chicago who revolutionized his field by quantitatively analyzing the impact of government economic regulations (previous economists’ analysis had focused on qualitative effects and theory). [1] In 1945, Stigler gathered data on a basic list of foods, nutrients, and prices, then played with the numbers to quantify the cheapest possible diet on the market. His work would inspire George Dantzig to develop the Simplex Algorithm just a few years later, one of the most efficient algorithms available today for solving general problems with linear constraint and objective functions. [2]

Stigler’s diet provided a very good baseline for understanding the price associated with a barebones minimalist diet. However, he didn’t take into account that food has to be bought in unit quantities (he considered only how many nutrients were contained in one dollar’s worth of flour, for instance) and he didn’t consider several additional constraints, like travel time and expense, differing food price/availability by store, and age and gender concerns. In our project, we sought to address both of these issues to provide a more complete picture of modern Americans’ access to good, nutritious food.

We got our nutrient data from the USDA’s Food Composition databases, available online. [3] We hypothesized different prices for different stores that varied based on season; we also created some estimate figures for gas prices and the like. In this report, we explain the theory behind the model in the Mathematical Model section and give the code we used in the code section. We discuss the key results immediately afterwards and provide some conclusions and directions for future study in the final section. We hope that this project will provide you the reader with a quantitative understanding of the tradeoffs between the best prices available and the time needed to travel to the stores that carry those best prices.

[1] http://www.econlib.org/library/Enc/bios/Stigler.html 
[2] https://research.googleblog.com/2014/09/sudoku-linear-optimization-and-ten-cent.html and https://web.archive.org/web/20160411141356/https://dl.dropboxusercontent.com/u/5317066/1990-dantzig-dietproblem.pdf 
[3] https://ndb.nal.usda.gov/ndb/search/list

## 2. Mathematical model ##

## General Assumptions

- Food prices are stable throughout the year and are not affected by seasonal variation and availability.
- 

#### Age Groups:

#### Stores:

#### Food costs:

#### Travel distance:

#### Travel cost:


## Scenario 1
$\text{There will be only one source of food and only one age group (adult).}$ 
$\text{Also, it is assumed that there is no additional cost in regards to transportation and that the climate remains constant.}$
<br></br>


$\textbf{Parameter definitions:}$

$F$ = number of foods  

$N$ = number of nutrients  

$f[i,j]$ = amount of nutrient *j* in food *i*  

$n[i]$ = minimum daily requirement for nutrient *i*  

$m[i]$ = maximum daily requirement for nutrient *i*  

$o[i]$ = maximum food limit for food *i*

$p[i]$ = price of food *i*

<br></br>

$\textbf{Variable definitions:}$  

$x[i]$ = number of units of food i purchased  
<br></br>

$\textbf{Constraints:}$  
##### 1. The amount of any given nutrient is greater than or equal to its lower limit and less than or equal to its upper limit.  
$$ \sum_{i=1}^{F} x[i]*f[i,j] \leq m[j] \text{, } \forall j \in N$$
$$ \sum_{i=1}^{F} x[i]*f[i,j] \geq n[j] \text{, } \forall j \in N$$  

##### 2. The amount of food should not exceed its upper limit.  
$$ x[i] \leq o[i] \text{, } \forall i \in F$$  

$\textbf{Objective:}$  
The objective is to minimize the total cost of food purchased.
$$ \text{Min } \sum_{i=1}^{F} x[i]*p[i] \text{, } \forall i \in F$$  

## Scenario 2
$\text{There will be multiple sources of food and three age groups, each with different nutrition requirements.}$ 
$\text{Also, it is assumed that there is no additional cost in regards to transportation.}$
<br></br>

$\textbf{*New* Parameter definitions (in addition to Scenario 1)}$

$A$ = number of age groups

$S$ = number of stores

$T$ = number of food categories 

$\textbf{*Updated* Parameter definitions (updated from Scenario 1)}$

$p[i,j]$ = price of food *i* at store *j*  

$n[i,a]$ = minimum daily requirement for nutrient *i* for age group *a*  

$m[i,a]$ = maximum daily requirement for nutrient *i* for age group *a*

<br></br>

$\textbf{Variable definitions: (updated from Scenario 1)}$  

$x[i,j,a]$ = number of units of food *i* purchased from store *j* for age group *a*
<br></br>

$\textbf{Constraints:}$  
##### 1. (updated) The amount of any given nutrient is greater than or equal to its lower limit and less than or equal to its upper limit for each age group 
$$ \sum_{i=1}^{F} x[i,j,a]*f[i,j] \leq m[j,a] \text{, } \forall j \in N \text{, } \forall a \in A$$
$$ \sum_{i=1}^{F} x[i,j,a]*f[i,j] \geq n[j,a] \text{, } \forall j \in N \text{, } \forall a \in A$$  

##### 2.  (new) The amount from each food category is less than or equal to its upper limit and at least one.  FIX 
$$\sum_{i=1}^{F} x[i] \leq p[i]$$  

$\textbf{Objective:}$  
The objective is to minimize the total cost of all food purchased from all stores.
$$ \text{Min } \sum_{i=1}^{F} x[i,j,a]*p[i,j] \text{, } \forall j \in S, \forall a \in A$$  

### Scenario 3 ###

This time, there is a travel cost associated with going to various locations as a tradeoff.  

$\textbf{*New* Parameter definitions (in addition to those from Scenario 2)}$

$Sd[i,j]$ = distance from location i to location j (in miles)  

$C$ = cost of gas per mile

$o[i,j]$ = order of stores traveled (binary matrix)

$\textbf{Constraints:}$  
- Add self-loop constraint
- Add single in and out edge constraint

$\textbf{Objective:}$  
$$ \text{Min } \sum_{a=1}^{A} \sum_{i=1}^{F} \sum_{j=1}^{F} x[i,j,a]*p[i,j] + \sum_{k=1}^{S} \sum_{l=1}^{S} o[k,l]*Sd[k,l]*C \text{, }$$  

## 3. Solution ##

### Spreadsheet Data

In [67]:
raw_food = readcsv("data/food_nutrients.csv")
raw_constraints = readcsv("data/constraints.csv")
raw_distances = readcsv("data/distances.csv")
raw_prices = readcsv("data/food_price.csv");

### Scenario 1:

In [87]:
using JuMP, Cbc, NamedArrays

(m,n) = size(raw_food)

###Data
num_nutrients = 17 #2:n-1
num_foods = 152 #17:m
food_categories = [:vegetables, :fruits, :grains, :meats, :dairy]

#This code converts the raw data from the spreadsheets into something that the Julia optimization model can use.
#First we need to get lists of available nutrients and foods.
nutrients = raw_food[1,:]  # the list of nutrients (convert to 1-D array)
foods = raw_food[:,1]            # the list of foods (convert to 1-D array)
#Here are the minimal and maximal amounts of nutrients associated with each age group.
child_minima = raw_constraints[4,:]
child_minima[2] = 1600
#println(child_minima)
child_maxima = raw_constraints[5,:]
child_maxima[2] = 1600
#println(child_maxima)
teen_minima = raw_constraints[7,:]
teen_minima[2] = 2400
#println(teen_minima)
teen_maxima = raw_constraints[8,:]
teen_maxima[2] = 2800
#println(teen_maxima)
adult_minima = raw_constraints[10,:]
adult_minima[2] = 2500
#println(adult_minima)
adult_maxima = raw_constraints[11,:]
adult_maxima[2] = 2500
#println(adult_maxima)

#Finally we need to interpret the food pricing data coming from the raw spreadsheet
println(foods)

#print(raw_prices[:,3:8])
foods_a = raw_food[3:154,1]
food_prices = hcat(foods_a,raw_prices[2:153,3:8])
println(food_prices)

max_quant = 10 #TODO: ADD TO SPREADSHEET
food_nutrients = NamedArray(raw_food[num_foods,num_nutrients], (foods,nutrients), ("foods","nutrients"));
food_prices = Array{Int64}(5) #Dict(zip[foods,raw_food[]]) #not added to spreadsheet yet

m = Model(solver=CbcSolver())

##Decision variable
@variable(m, food[1:num_food])

##Constraints

#Min/Max nutrients
@constraint(m, min_nutrient[j in 1:num_nutrients], sum(food[i] * food_nutrients[i,j] for i in 1:num_foods) >= min_req[j])
@constraint(m, max_nutrient[j in 1:num_nutrients], sum(food[i] * food_nutrients[i,j] for i in 1:num_foods) <= max_req[j])

#Max food amount
@constraint(m, max_food[i in 1:num_foods], sum(food[i] * max_quant[i]))

#Sum of each food category must be at least 1
#TODO

##Objective
@objective(m, Min, sum[food[i] * food_prices[i] for i in 1:num_foods])
solve(m)

Any["", "", "Butter", "Cheddar Cheese", "Gouda Cheese", "Mozzarella Cheese", "Muenster Cheese", "Parmesan Cheese", "American Cheese", "Swiss Cheese", "Eggnog", "1% Milk", "Vanilla yogurt", "Egg", "Chobani vanilla Greek yogurt", "Allspice", "Cardamom", "Basil", "Cinnamon", "Coriander", "Garlic", "Nutmeg", "Paprika", "Parsley", "Black Pepper", "Rosemary", "Thyme", "Sage", "Salt", "Vinegar", "Vanilla", "Chicken", "Duck", "Turkey", "Apple babyfood", "Bananas babyfood", "Infant formula", "Thousand island salad dressing", "Italian dressing", "Mayo", "Olive oil", "Gravy", "Tomato bisque soup", "Clam chowder", "Pesto sauce", "Worcestershire sauce", "Chicken broth", "Cream of chicken soup", "Chicken noodle soup", "Sandwich ham", "Pastrami", "Sandwich salami", "Sandwich bologna", "Cheerios", "Fruit and cream oatmeal", "Dinosaur eggs oatmeal", "Apples", "Apricots", "Bananas", "Blackberries", "Avocadoes", "Blueberries", "Cherries", "Cranberries", "Grapes", "Kiwi", "Florida oranges", "Papayas", "Pe

LoadError: [91mMethodError: no method matching NamedArrays.NamedArray(::Float64, ::Tuple{Array{Any,1},Array{Any,1}}, ::Tuple{String,String})[0m
Closest candidates are:
  NamedArrays.NamedArray([91m::AbstractArray{T,N}[39m, ::Tuple{Vararg{Array{T,1} where T,N}}, ::Tuple{Vararg{Any,N}}) where {T, N} at C:\Users\Galadriel\.julia\v0.6\NamedArrays\src\constructors.jl:33
  NamedArrays.NamedArray([91m::AbstractArray{T,N}[39m, [91m::Tuple{}[39m, ::Tuple{Vararg{Any,N}}) where {T, N} at C:\Users\Galadriel\.julia\v0.6\NamedArrays\src\constructors.jl:22
  NamedArrays.NamedArray([91m::AbstractArray{T,N}[39m, [91m::Tuple{Vararg{DataStructures.OrderedDict,N}}[39m, ::Tuple{Vararg{Any,N}}) where {S<:NamedArrays.NamedArray, T, N} at C:\Users\Galadriel\.julia\v0.6\NamedArrays\src\namedarraytypes.jl:21
  ...[39m

### Scenario 2

In [ ]:
using JuMP, Cbc, NamedArrays

(m,n) = size(raw_food)

###Data
num_nutrients = 17 #2:n-1
num_foods = 152 #17:m
food_categories = [:vegetables, :fruits, :grains, :meats, :dairy]


nutrients = raw_food[1,num_nutrients][:]   # the list of nutrients (convert to 1-D array)
foods = raw_food[num_foods,1][:]             # the list of foods (convert to 1-D array)
#min_req = Dict(zip(nutrients,raw[10,num_nutrients]))
#max_req = Dict(zip(nutrients,raw[11,num_nutrients]))
max_quant = 10 #TODO: ADD TO SPREADSHEET
food_nutrients = NamedArray(raw_food[num_foods,num_nutrients], (foods,nutrients), ("foods","nutrients"));
food_prices = Array{Int64}(5) #Dict(zip[foods,raw_food[]]) #not added to spreadsheet yet

m = Model(solver=CbcSolver())

##Decision varaible
@variable(m, food[1:num_food])

##Constraints

#Min/Max nutrients
@constraint(m, min_nutrient[j in 1:num_nutrients], sum(food[i] * food_nutrients[i,j] for i in 1:num_foods) >= min_req[j])
@constraint(m, max_nutrient[j in 1:num_nutrients], sum(food[i] * food_nutrients[i,j] for i in 1:num_foods) <= max_req[j])

#Max food amount
@constraint(m, max_food[i in 1:num_foods], sum(food[i] * max_quant[i]))

#Sum of each food category must be at least 1
#TODO

##Objective
@objective(m, Min, sum[food[i] * food_prices[i] for i in 1:num_foods])
solve(m)

### Scenario 3 (Full Model)

In [ ]:
using JuMP, Cbc

raw = readcsv("food_data.csv")
(m,n) = size(raw)

###Data
num_nutrients = 2:n
num_food = 17:m 
food_categories = [:vegetables, :fruits, :grains, :meats, :dairy]
age_groups = [:child, :teenager, :adult]
locations = #add locations
gas = 3

nutrients = raw[1,num_nutrients][:]   # the list of nutrients (convert to 1-D array)
foods = raw[num_foods,1][:]             # the list of foods (convert to 1-D array)
min_req = Dict(zip(nutrients,raw[10,num_nutrients]))
max_req = Dict(zip(nutrients,raw[11,num_nutrients]))
max_quant = #not added to spreadsheet yet
food_nutrients = NamedArray(raw[num_food,num_nutrient], (foods,nutrients), ("foods","nutrients"));
food_prices = #not added to spreadsheet yet
distances = #no data yet


m = Model(solver=CbcSolver())

##Decision varaible
@variable(m, food[1:num_food])
@variable(m, path[i in locations, j in locations])

##Constraints

#Min/Max nutrients
@constraint(m, min_nutrient[j in 1:num_nutrients], sum(food[i] * food_nutrients[i,j] for i in 1:num_foods) >= min_req[j])
@constraint(m, max_nutrient[j in 1:num_nutrients], sum(food[i] * food_nutrients[i,j] for i in 1:num_foods) <= max_req[j])

#Max food amount
@constraint(m, max_food[i in 1:num_foods], sum(food[i] * max_quant[i]))

#Sum of each food category must be at least 1
#TODO

##Objective
@objective(m, Min, sum[food[i] * food_prices[i] for i in 1:num_foods] + sum(path[k,l] * distances[k,l] * gas for k in 1 in locations, l in locations))
solve(m)

Remember to make sure your code compiles! I will be running your code!

## 4. Results and discussion ##

Here, you display and discuss the results. Show figures, plots, images, trade-off curves, or whatever else you can think of to best illustrate your results. The discussion should explain what the results mean, and how to interpret them. You should also explain the limitations of your approach/model and how sensitive your results are to the assumptions you made.

Use plots (see `PyPlot` examples from class), or you can display results in a table like this:

| Tables        | Are           | Cool  |
| ------------- |:-------------:| -----:|
| col 3 is      | right-aligned |\$1600 |
| col 2 is      | centered      |  \$12 |
| zebra stripes | are neat      |   \$1 |

### 4.A. Feel free to add subsections

#### 4.A.a. or subsubsections

## 5. Conclusion ##

Summarize your findings and your results, and talk about at least one possible future direction; something that might be interesting to pursue as a follow-up to your project.

### TODO:
   - Price scaling factor
   - Add taste factor?

Annie
   - Add words for data section
   - Add max quantity constraint data to spreadsheet
   
Harish
   - Code out three models
   - Update math models for three scenarios
   
Idea?
   - Scenario 4: taste factor
   
   
Section 4:
- Pareto curve